#  Optimize my USA trip

## Capstone project submission for IBM Data Science professional certification
### submitted by : Maneesh Hari Disawal

### Problem statement and background :
__"Once you have seen a beach, you have seen them all"__, that is what they say.

I'm planning to visit the United States of America next year. However, I have a limit budget and time. With that limitation, I want to maximize my experiences and visit the most variety of the states of America.
Through this project, I'll be trying to group together similar states, based on the tourist attractions for each state. Once an optimal number of groups of states have been calculated, one representative state from each group shall be selected as the state to visit to experience that group.

### Description of data
Data from FourSquare.com will be used for the purposes of this project.
A list of top n (exact number to be confirmed based on the query limitations) tourist attractions will be queried for each state, with the following attributes:
1. Type of attraction
2. Average rating
3. Representative location (assumed to be the location of the top rated attraction)
This is based on the initial understanding. When the actual data is extracted and explored, some other parameters may become available and might be incorporated as well.

### Methodology
Basic assumption is "A beach, is a beach, is a beach". 

In other words, one pre-historic museum is similar to any other pre-historic museum and one walking trail is same as any other. Thus, maximising the types of attractions visited.

A clustering algorithem shall be implemented to statistically group the states together based on the available data.

### Known limitations
It is acknowledged that this system has, at least, the following limitations:
1. All states are treated equally irrespective of their sizes, i.e. To begin with Rhode Island has the same weightage as California.
2. The currect method does not account for factors like, cost of travel in different states, weather at the time of travel etc.

Import necessary libraries

In [109]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs 
%matplotlib inline

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.19.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Folium installed
Libraries imported.


In [2]:
CLIENT_ID = 'BRKWUY3N53OVHDZY0I1F2NL0I0S4VDRFWV21NHANRBOP2U1T' # your Foursquare ID
CLIENT_SECRET = 'YMGTHBXJYPKXMIOME2RUI0YGI3RITCEH25K2Q3F4RHNAJT3Q' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
QSECTION = 'sights'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BRKWUY3N53OVHDZY0I1F2NL0I0S4VDRFWV21NHANRBOP2U1T
CLIENT_SECRET:YMGTHBXJYPKXMIOME2RUI0YGI3RITCEH25K2Q3F4RHNAJT3Q


In [3]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&near={}&v={}&section={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, 'NY,USA', VERSION, QSECTION, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=BRKWUY3N53OVHDZY0I1F2NL0I0S4VDRFWV21NHANRBOP2U1T&client_secret=YMGTHBXJYPKXMIOME2RUI0YGI3RITCEH25K2Q3F4RHNAJT3Q&near=NY,USA&v=20180604&section=sights&limit=100'

In [4]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cab8a084c1f6715e17942e4'},
 'response': {'confident': False,
  'geocode': {'feature': {'cc': 'US',
    'displayName': 'New York, NY, United States',
    'geometry': {'bounds': {'ne': {'lat': 40.882214, 'lng': -73.907},
      'sw': {'lat': 40.679548, 'lng': -74.047285}},
     'center': {'lat': 40.742185, 'lng': -73.992602}},
    'highlightedName': '<b>NY</b>, <b>USA</b>',
    'id': 'geonameid:5128581',
    'longId': '72057594043056517',
    'matchedName': 'NY, USA',
    'name': 'New York',
    'slug': 'new-york-city-new-york',
    'woeType': 7},
   'parents': [],
   'what': '',
   'where': 'ny usa'},
  'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1d3941735',
      'name': 'Vegetarian / Vegan Restaurant',
      'pluralName': 'Vegetarian / Vegan Restaurants',
      'primary': True,
      'shortName': 'Vegetarian / Vegan'}],
    'delivery'

Thought process for data processing = a rough sketch
- Get the list of venues by categories by state, e.g. 10 Parks, 5 museums, 3 churches etc.
- Explore the data and limit the number of categories with a reasonable number of entities. 
- The above condition will simplify the analysis but has a drawback of actually punishing the unique attractions, i.e. if there is only one "Biggest waterfall in the world" then that would be grouped into others.
- Infact, as I think about it, the "others" categories might be the unique attractions and can play with the weightage to get clusters of states with most widely different attractions
- Normalize data for comparison. Need to check the algorithm to do it but I imagine doing % by categories
- apply a clustering algorithm for a range of K's and plot the results in a graph to arrive at an optimal value.
- within each cluster, calculate average ratings for all reported venues and select the top states.
- that is the recommendations

In [5]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_d5e9c96d810a4965bbc441cf17c63de7 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='u7nMOGD8F_NCnEwMZBW8YfnxgLHDdPwNykxX7CuVEStg',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_d5e9c96d810a4965bbc441cf17c63de7.get_object(Bucket='mlpythonfinalproject-donotdelete-pr-uchda6nsi7mhen',Key='USA_50_States.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_states = pd.read_csv(body)
df_states.head()



,StateName,StateAbbr
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [71]:
Master_Categories = {}
STATE_CAT_MAP = [] # list to store all categories data

for index, row in df_states.iterrows():
    STATENAME = row['StateAbbr']+',USA'
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&near={}&v={}&section={}'.format(CLIENT_ID, CLIENT_SECRET, STATENAME, VERSION, QSECTION)
    results = requests.get(url).json()
    x=0
    for data in results['response']['venues']:
        try:
            curr_cat =  results['response']['venues'][x]['categories'][0]['name']
        except IndexError as e:
            a=1
        STATE_CAT_MAP.append([row['StateAbbr'],curr_cat,1])
        if curr_cat in Master_Categories:
            Master_Categories[curr_cat]=Master_Categories[curr_cat]+1
        else:
            Master_Categories[curr_cat]=1
        x=x+1
STATE_CAT_MAP

[['AL', 'Ice Cream Shop', 1],
 ['AL', 'Ice Cream Shop', 1],
 ['AL', 'Church', 1],
 ['AL', 'Gas Station', 1],
 ['AL', 'Hotel', 1],
 ['AL', 'Church', 1],
 ['AL', 'Intersection', 1],
 ['AL', 'City', 1],
 ['AL', 'Bridge', 1],
 ['AL', 'Garden Center', 1],
 ['AL', 'Athletics & Sports', 1],
 ['AL', 'Gym', 1],
 ['AL', 'Flower Shop', 1],
 ['AL', 'Automotive Shop', 1],
 ['AL', 'Radio Station', 1],
 ['AL', 'Convenience Store', 1],
 ['AL', 'Post Office', 1],
 ['AL', 'High School', 1],
 ['AL', 'Gas Station', 1],
 ['AL', 'Farm', 1],
 ['AL', 'Farm', 1],
 ['AL', 'Church', 1],
 ['AL', 'Church', 1],
 ['AL', 'Fire Station', 1],
 ['AL', 'Fire Station', 1],
 ['AL', 'Fire Station', 1],
 ['AL', 'Parking', 1],
 ['AL', 'Church', 1],
 ['AL', 'Professional & Other Places', 1],
 ['AL', 'Factory', 1],
 ['AZ', 'Car Wash', 1],
 ['AZ', 'Bed & Breakfast', 1],
 ['AZ', 'Trailer Park', 1],
 ['AZ', 'Building', 1],
 ['AZ', 'Building', 1],
 ['AZ', 'Building', 1],
 ['AZ', 'General Travel', 1],
 ['AZ', 'General Travel', 1],
 

### Get the list of Top 25 categories

In [104]:
top25Cats=[] # placeholder for top 25 categories
listofTuples = sorted(Master_Categories.items() , reverse=True,  key=lambda x: x[1])
x=0
# Iterate over the sorted sequence
for elem in listofTuples :
    if x < 25:
        top25Cats.append(elem[0])
    x=x+1

top25Cats

['Building',
 'Gas Station',
 'Farm',
 'Church',
 'City',
 'Other Great Outdoors',
 'American Restaurant',
 'Campground',
 'Automotive Shop',
 'Office',
 'Bar',
 'Scenic Lookout',
 'Convenience Store',
 'Bank',
 'Field',
 'Pizza Place',
 'Salon / Barbershop',
 'Government Building',
 'Financial or Legal Service',
 'Storage Facility',
 'Capitol Building',
 'Miscellaneous Shop',
 'Gym',
 'Trail',
 'Lake']

In [105]:
# Summarize and Pivot the data
df2 = pd.DataFrame(np.array(STATE_CAT_MAP), columns=(['St','Cat','Cnt']))
df3 = df2.groupby(['St','Cat'],as_index=False)['Cnt'].count()
df4 = df3.pivot(index='St',columns='Cat',values='Cnt')


## That is a total of 45 states' data across 280 categories

### Limiting to top 25 categories only

In [108]:
# limiting to our top 25 categories only
df5 = df4[top25Cats]
df5.head()

Cat,Building,Gas Station,Farm,Church,City,Other Great Outdoors,American Restaurant,Campground,Automotive Shop,Office,...,Pizza Place,Salon / Barbershop,Government Building,Financial or Legal Service,Storage Facility,Capitol Building,Miscellaneous Shop,Gym,Trail,Lake
St,,,,,,,,,,,,,,,,,,,,,
AL,NaN,2.0,2.0,5.0,1.0,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
AR,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZ,3.0,NaN,NaN,NaN,1.0,1.0,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CA,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,2.0,NaN,...,1.0,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,NaN
CO,NaN,NaN,NaN,1.0,NaN,4.0,NaN,3.0,1.0,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,3.0


In [107]:
df5.shape

(45, 25)

In [111]:
# Update NaN values and normalize data
from sklearn.preprocessing import StandardScaler
X = df5.values[:,1:]
X = np.nan_to_num(X)
Clus_dataSet = StandardScaler().fit_transform(X)
Clus_dataSet

array([[ 1.30182256,  0.87237645,  3.43595365, ...,  1.27872403,
        -0.4125685 , -0.43643578],
       [ 0.30890705, -0.45836729, -0.5083809 , ..., -0.46499055,
        -0.4125685 , -0.43643578],
       [-0.68400846, -0.45836729, -0.5083809 , ..., -0.46499055,
        -0.4125685 , -0.43643578],
       ..., 
       [ 0.30890705, -0.45836729,  0.28048601, ..., -0.46499055,
        -0.4125685 , -0.43643578],
       [ 0.30890705, -0.45836729,  1.06935292, ...,  1.27872403,
         1.13456337, -0.43643578],
       [ 0.30890705, -0.45836729, -0.5083809 , ..., -0.46499055,
        -0.4125685 ,  1.2001984 ]])

### Applying K-means on the data set. Starting with a K = 5

In [142]:
clusterNum = 8
k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 12)
k_means.fit(X)
labels = k_means.labels_
print(labels)

[0 6 5 6 5 6 5 6 6 6 6 5 3 6 0 6 6 0 6 6 3 6 3 2 3 5 6 1 6 1 6 5 4 6 0 0 3
 1 7 0 6 1 6 0 6]


In [143]:
# Assign lables to the dataset
df5["SightCluster"] = labels
df5.head(5)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Cat,Building,Gas Station,Farm,Church,City,Other Great Outdoors,American Restaurant,Campground,Automotive Shop,Office,...,Government Building,Financial or Legal Service,Storage Facility,Capitol Building,Miscellaneous Shop,Gym,Trail,Lake,Clus_km,SightCluster
St,,,,,,,,,,,,,,,,,,,,,
AL,NaN,2.0,2.0,5.0,1.0,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,4,0
AR,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,6
AZ,3.0,NaN,NaN,NaN,1.0,1.0,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,5
CA,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,2.0,NaN,...,NaN,2.0,NaN,NaN,2.0,NaN,NaN,NaN,0,6
CO,NaN,NaN,NaN,1.0,NaN,4.0,NaN,3.0,1.0,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,1.0,3.0,1,5


In [144]:
# Get grouping and Statename
df6 = df5.loc[:,'SightCluster']
df6

St
AL    0
AR    6
AZ    5
CA    6
CO    5
CT    6
DE    5
FL    6
GA    6
HI    6
IA    6
ID    5
IL    3
IN    6
KS    0
KY    6
LA    6
MA    0
MD    6
ME    6
MI    3
MN    6
MO    3
MS    2
MT    3
NC    5
ND    6
NE    1
NH    6
NJ    1
NY    6
OH    5
OK    4
PA    6
RI    0
SC    0
SD    3
TN    1
TX    7
UT    0
VA    6
VT    1
WA    6
WI    0
WV    6
Name: SightCluster, dtype: int32

In [145]:
ST_Clus=[] # placeholder for Clustered states
listofTuples = sorted(df6.items() , reverse=True,  key=lambda x: x[1])
# Iterate over the sorted sequence
for elem in listofTuples :
    ST_Clus.append([elem[0],elem[1]])

ST_Clus

[['TX', 7],
 ['AR', 6],
 ['CA', 6],
 ['CT', 6],
 ['FL', 6],
 ['GA', 6],
 ['HI', 6],
 ['IA', 6],
 ['IN', 6],
 ['KY', 6],
 ['LA', 6],
 ['MD', 6],
 ['ME', 6],
 ['MN', 6],
 ['ND', 6],
 ['NH', 6],
 ['NY', 6],
 ['PA', 6],
 ['VA', 6],
 ['WA', 6],
 ['WV', 6],
 ['AZ', 5],
 ['CO', 5],
 ['DE', 5],
 ['ID', 5],
 ['NC', 5],
 ['OH', 5],
 ['OK', 4],
 ['IL', 3],
 ['MI', 3],
 ['MO', 3],
 ['MT', 3],
 ['SD', 3],
 ['MS', 2],
 ['NE', 1],
 ['NJ', 1],
 ['TN', 1],
 ['VT', 1],
 ['AL', 0],
 ['KS', 0],
 ['MA', 0],
 ['RI', 0],
 ['SC', 0],
 ['UT', 0],
 ['WI', 0]]

### Thus we have reduced the 50 states into 8 groups of states, purely based on the FourSquare data of the top sights in the state.
### Also, only considering the top 25 categories of the overall USA

### We see that groupings 2,4, and 7 have one state each.
### Indicating the variety of sights offered in each of these states.
## Overall recommendation would be to visit the following states
1. Texas,
2. Oklahoma, and 
3. Missisippi

Post that to chose one state from the rest of the groups.

### Thank you very much for reviewing my work.
### Any feedback is welcome. I'll continue to improve this algorithm with additional data and constraints post the program.